# Vibration Data Analysis with Plotly

## Setting things

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
import ipywidgets as widgets
from IPython.display import display
import tkinter as tk
from tkinter.filedialog import askopenfilename
import numpy as np
from scipy.signal import correlate
import os

def is_vscode():
    """
    Checks if the Jupyter notebook is running inside VSCode.

    Returns
    -------
    bool
        True if running inside VSCode, False otherwise.
    """
    return 'VSCODE_PID' in os.environ

import plotly.io as pio
# print(pio.renderers)
if is_vscode():
    pio.renderers.default = 'vscode'
else:
    pio.renderers.default = 'iframe'


# Dictionary to store the SQLite connection file path
db_connection = {}

def select_sqlite_file():
    """
    Opens a file selection dialog via a GUI button in Jupyter Notebook for the user 
    to select an SQLite database file and establishes a connection.

    The function displays an interactive button using `ipywidgets`. 
    When clicked, it opens a Tkinter-based file dialog to select a `.sqlite` file. 
    After a successful file selection, it connects to the database and stores the 
    connection object in the global `db_connection` dictionary under the key `'conn'`.

    Notes
    -----
    - This function is designed to work in Jupyter Notebook environments.
    - The connection object can be retrieved after selection using:
      `conn = db_connection.get('conn')`
    - If the user cancels the selection, no connection is established.

    Examples
    --------
    >>> select_sqlite_file()
    # Displays a button. After clicking and selecting a file:
    >>> conn = db_connection.get('conn')
    >>> cursor = conn.cursor()
    >>> cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    >>> cursor.fetchall()
    """
    def on_button_click(b):
        root = tk.Tk()
        root.withdraw()
        file_path = askopenfilename(
            title="Select SQLite database file",
            filetypes=[("SQLite Database Files", "*.sqlite"), ("All Files", "*.*")]
        )
        if file_path:
            
            db_connection['file_path'] = file_path
            print(f"✅ Connected to database: {file_path}")
        else:
            print("⚠️ No file selected.")

    button = widgets.Button(description="Select SQLite Database")
    button.on_click(on_button_click)
    display(button)

## Reading database

In [2]:
select_sqlite_file()

Button(description='Select SQLite Database', style=ButtonStyle())

In [4]:
conn = conn = sqlite3.connect(db_connection["file_path"])
print("File path:\n", db_connection["file_path"])
df = pd.read_sql_query('SELECT * FROM vibration_samples', conn)
conn.close()

File path:
 C:/Users/EMKA/OneDrive - PETROBRAS/10 - GitHub/RealTimeModalAnalysis/vibration_data_20250321_115602.sqlite


### Processing 

In [5]:
df['datetime'] = pd.to_datetime(df['abs_timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')

## Inspecting database

In [23]:
display(df)

,abs_timestamp,timestamp,reference,x,y,z,datetime
0,1.742569e+09,50.160,-0.006865,-0.001492,0.001858,0.001534,2025-03-21 11:55:52.378349781-03:00
1,1.742569e+09,50.161,-0.007603,-0.001254,0.001992,0.001437,2025-03-21 11:55:52.379349947-03:00
2,1.742569e+09,50.162,-0.008121,-0.001498,0.001583,0.000949,2025-03-21 11:55:52.380349874-03:00
3,1.742569e+09,50.163,-0.007908,-0.002235,0.001467,0.000736,2025-03-21 11:55:52.381350040-03:00
4,1.742569e+09,50.164,-0.007926,-0.001839,0.001718,0.001339,2025-03-21 11:55:52.382350206-03:00
...,...,...,...,...,...,...,...
9995,1.742569e+09,60.156,-0.005535,-0.001486,0.003177,0.001705,2025-03-21 11:56:02.374349356-03:00
9996,1.742569e+09,60.157,-0.006023,-0.001041,0.002786,0.001339,2025-03-21 11:56:02.375349522-03:00
9997,1.742569e+09,60.158,-0.006657,-0.000803,0.002188,0.001656,2025-03-21 11:56:02.376349688-03:00
9998,1.742569e+09,60.159,-0.006468,-0.000632,0.001485,0.001992,2025-03-21 11:56:02.377349615-03:00


## Plot Time Series for All Axis

In [6]:
fig = px.line(df, x='datetime', y=['reference', 'x', 'y', 'z'],
              title='Time Series of Vibration Data')
fig.show()

## Plot FFT of Each Axis

In [ ]:
sampling_rate = 1000  # Hz
fft_results = {}
for col in ['reference', 'x', 'y', 'z']:
    fft_values = np.abs(np.fft.rfft(df[col].values - df[col].mean()))
    fft_freqs = np.fft.rfftfreq(len(df[col]), d=1/sampling_rate)
    fft_results[col] = (fft_freqs, fft_values)
fig = go.Figure()
for axis, (freqs, fft_vals) in fft_results.items():
    fig.add_trace(go.Scatter(x=freqs, y=fft_vals, mode='lines', name=axis))
    fig.update_layout(title=f'FFT of all channels', xaxis_title='Frequency (Hz)', yaxis_title='Amplitude')
fig.show()

## Correlation Between Reference and Each Axis

In [ ]:

lags = np.arange(-len(df)//2, len(df)//2) / sampling_rate
fig = go.Figure()
for axis in ['x', 'y', 'z']:
    corr = correlate(df[axis].values - df[axis].mean(), df['reference'].values - df['reference'].mean(), mode='full')
    
    fig.add_trace(go.Scatter(x=lags, y=corr, mode='lines', name=f'Correlation with {axis}'))
    fig.update_layout(title=f'Cross-Correlation between Reference', xaxis_title='Lag (s)', yaxis_title='Correlation')
fig.show()
